In [7]:
import numpy as np

def binomial_option_pricing(S0, K, r, T, sigma, N, option_type="European"):
    """
    Binomial option pricing model for European and American put options.

    Parameters:
        S0 (float): Initial stock price
        K (float): Strike price
        r (float): Risk-free interest rate
        T (float): Time to maturity
        sigma (float): Volatility
        N (int): Number of steps in the binomial tree
        option_type (str): "European" or "American"

    Returns:
        float: Option price
    """
    # Calculate parameters
    dt = T / N
    u = np.exp(sigma * np.sqrt(dt))  # Up factor
    d = np.exp(-sigma * np.sqrt(dt))  # Down factor
    p = (np.exp(r * dt) - d) / (u - d)  # Risk-neutral probability
    discount = np.exp(-r * dt)  # Discount factor

    # Initialize stock prices at maturity
    asset_prices = np.zeros(N + 1)
    for i in range(N + 1):
        asset_prices[i] = S0 * (u ** i) * (d ** (N - i))

    # Calculate terminal payoffs
    option_values = np.maximum(K - asset_prices, 0)  # Put option payoff

    # Backward induction
    for step in range(N - 1, -1, -1):
        for i in range(step + 1):
            continuation_value = discount * (p * option_values[i + 1] + (1 - p) * option_values[i])
            if option_type == "American":
                # Early exercise value
                exercise_value = K - (S0 * (u ** i) * (d ** (step - i)))
                option_values[i] = max(continuation_value, exercise_value)
            else:
                option_values[i] = continuation_value

    return option_values[0]


# Parameters for Problem 2
S0 = 100     # Initial stock price
K = 105      # Strike price
r = 0.05     # Risk-free interest rate
T = 0.5      # Time to maturity
sigma = 0.445  # Volatility
N = 100      # Number of steps

# European put option
european_price = binomial_option_pricing(S0, K, r, T, sigma, N, option_type="European")
print(f"European Put Option Price (N = {N}): {european_price:.2f}")

# American put option
american_price = binomial_option_pricing(S0, K, r, T, sigma, N, option_type="American")
print(f"American Put Option Price (N = {N}): {american_price:.2f}")

# Convergence analysis for American put option
tolerance = 0.01
price_difference = float("inf")
steps = 100

while price_difference > tolerance:
    price_current = binomial_option_pricing(S0, K, r, T, sigma, steps, option_type="American")
    price_next = binomial_option_pricing(S0, K, r, T, sigma, steps + 50, option_type="American")
    price_difference = abs(price_next - price_current)
    steps += 1

print(f"American Put Option Price (Converged): {price_next:.2f}")
print(f"Steps for Convergence: {steps}")


European Put Option Price (N = 100): 13.90
American Put Option Price (N = 100): 14.21
American Put Option Price (Converged): 14.22
Steps for Convergence: 102
